# Tutorial: eQTL Analysis with `SAIGE-QTL` using cellink
This tutorial demonstrates how to perform eQTL analysis using `SAIGE-QTL` through the cellink package. `SAIGE-QTL` uses a Poisson mixed model that accounts for sample relatedness and is particularly well-suited for single-cell eQTL analysis. It also accounts for sample relatedness through genetic relationship matrices, handles both common and rare variant analysis and supports cell-level and individual-level covariates. This notebook assumes familiarity with single-cell data processing and basic statistical genetics concepts. The cellink package provides convenient wrapper functions that handle data preparation and formatting for `SAIGE-QTL`. For the installation please refer to the docs of [`SAIGE-QTL`](https://weizhou0.github.io/SAIGE-QTL-doc/docs/Installation.html). We recommend using the provided [Docker container](https://hub.docker.com/r/wzhou88/saigeqtl).

SAIGE-QTL analysis consists of three steps:
- Step 1: Fit null Poisson mixed model (one per gene)
- Step 2: Perform association tests (single-variant or set-based)
- Step 3 (optional): Calculate gene-level p-values using ACAT

SAIGE-QTL has a range of further options, such as set-based analysis, genome-wide analysis with LOCO or conditional analysis, which are alll supported in the `cellink` wrapper. For more details please however refer to the docs of [`SAIGE-QTL`](https://weizhou0.github.io/SAIGE-QTL-doc/docs/Installation.html).

## Environment Setup

In [1]:
import scanpy as sc

from cellink.resources import get_dummy_onek1k
from cellink.tl.external import run_saigeqtl, configure_saigeqtl_runner

# Analysis parameters
n_gpcs = 20
n_epcs = 15
chrom = 22
cis_window = 500_000
cell_type = "CD8 Naive"
celltype_key = "predicted.celltype.l2"

## Configure SAIGE-QTL Runner
The runner handles execution across different environments (local, docker, singularity):

In [2]:
runner = configure_saigeqtl_runner("../../src/cellink/tl/external/config/saigeqtl_docker.yaml")

## Load and Prepare Data

In [3]:
dd = get_dummy_onek1k(config_path="../../src/cellink/resources/config/dummy_onek1k.yaml", verify_checksum=False)
print(f"Dataset shape: {dd.shape}")

dd.G.obsm["gPCs"] = dd.G.obsm["gPCs"][dd.G.obsm["gPCs"].columns[:n_gpcs]]

dd.aggregate(obs=["donor_id", "sex", "age"], func="first", add_to_obs=True)

dd = dd[..., dd.C.obs[celltype_key] == cell_type, :].copy()
print(f"After cell type filtering: {dd.shape}")

dd.G.obs["donor_sex"] = dd.G.obs["sex"]
dd.G.obs["donor_age"] = dd.G.obs["age"]

dd = dd.sel(G_var=dd.G.var.chrom == str(chrom), C_var=dd.C.var.chrom == str(chrom)).copy()
print(f"After chromosome {chrom} filtering: {dd.shape}")

dd = dd[:, dd.G.var["pos"] < 17584955, :, :].copy()

[2026-01-09 01:39:30,021] INFO:root: /Users/larnoldt/cellink_data/dummy_onek1k/dummy_onek1k.dd.h5 already exists
[2026-01-09 01:39:30,022] WARNING:root: No checksum provided, skipping verification
[2026-01-09 01:39:31,204] INFO:root: Loaded dummy OneK1K dataset: (100, 146939, 125366, 34073)
Dataset shape: (100, 146939, 125366, 34073)
After cell type filtering: (100, 146939, 4756, 34073)
After chromosome 22 filtering: (100, 136776, 4756, 871)


## Analysis Workflows
### 1. Basic cis-eQTL Analysis (Single-Variant Tests, Steps 1 + 2)
The most common use case: fit null model and run association tests.

In [4]:
results = run_saigeqtl(
    dd,
    gene_col="ENSG00000273362",  # Gene ID from expression data
    prefix="saigeqtl_complete",
    steps=[1, 2],
    mode="cis",
    analysis_type="single_variant",
    window=cis_window,
    min_mac=20,
    sample_covariates=["gPCs"],
    use_grm_to_fit_null=False,
    overwrite_variance_ratio_file=True,
    run=True,
)

results.head()

Writing BED: 100%|██████████| 1/1 [00:00<00:00, 147.93it/s]

Writing FAM... done.
Writing BIM... done.
[2026-01-09 01:39:32,509] INFO:cellink.tl.external._saigeqtl: Running Step 1: Fitting null model
[2026-01-09 01:39:32,510] INFO:cellink.tl._runner: Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data wzhou88/saigeqtl /app/.pixi/envs/default/bin/Rscript /app/extdata/step1_fitNULLGLMM_qtl.R --phenoFile=saigeqtl_complete_phenotype.txt --phenoCol=ENSG00000273362 --sampleIDColinphenoFile=IND_ID --traitType=count --plinkFile=saigeqtl_complete --outputPrefix=saigeqtl_complete --LOCO=FALSE --useGRMtoFitNULL=FALSE --IsOverwriteVarianceRatioFile=TRUE --covarColList=sex,age,gPCs_0,gPCs_1,gPCs_2,gPCs_3,gPCs_4,gPCs_5,gPCs_6,gPCs_7,gPCs_8,gPCs_9,gPCs_10,gPCs_11,gPCs_12,gPCs_13,gPCs_14,gPCs_15,gPCs_16,gPCs_17,gPCs_18,gPCs_19 --sampleCovarColList=sex,age,gPCs_0,gPCs_1,gPCs_2,gPCs_3,gPCs_4,gPCs_5,gPCs_6,gPCs_7,gPCs_8,gPCs_9,gPCs_10,gPCs_11,gPCs_12,gPCs_13,gPCs_14,gPCs_15,gPCs_16,g

[2026-01-09 01:39:49,461] INFO:cellink.tl._runner: R version 4.4.3 (2025-02-28)
Platform: x86_64-conda-linux-gnu
Running under: Ubuntu 20.04.4 LTS

Matrix products: default
BLAS/LAPACK: /app/.pixi/envs/default/lib/libopenblasp-r0.3.30.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: NA
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] optparse_1.7.5 SAIGEQTL_0.3.2

loaded via a namespace (and not attached):
[1] compiler_4.4.3      Matrix_1.7-3        Rcpp_1.1.0         
[4] getopt_1.20.4       RcppNumerical_0.6-0 grid_4.4.3         
[7] data.table_1.17.6   RcppParallel_5.1.9  lattice_0.22-

,CHR,POS,MarkerID,Allele1,Allele2,AC_Allele2,AF_Allele2,MissingRate,BETA,SE,Tstat,var,p.value,p.value.NA,Is.SPA,N
0,22,16569715,22_16569715_G_A,A,G,106,0.530,0,9.658680e-16,54135.1,3.295800e-25,3.412270e-10,1,1,False,100
1,22,16569887,22_16569887_T_A,A,T,133,0.665,0,7.073560e-16,61436.3,1.874080e-25,2.649420e-10,1,1,False,100
2,22,16570090,22_16570090_ACATT_A,A,ACATT,151,0.755,0,5.051070e-16,69893.4,1.033980e-25,2.047040e-10,1,1,False,100
3,22,16570885,22_16570885_T_G,G,T,141,0.705,0,1.208410e-15,63757.8,2.972680e-25,2.459990e-10,1,1,False,100
4,22,16571233,22_16571233_G_A,A,G,101,0.505,0,5.824050e-16,56733.4,1.809460e-25,3.106870e-10,1,1,False,100


### 2. Step-by-Step Analysis
Run each step independently for more control:
#### Step 1 Only: Fit Null Model

In [5]:
step1_results = run_saigeqtl(
    dd,
    gene_col="ENSG00000273362",
    prefix="gene1_step1",
    steps=[1],  # Only Step 1
    sample_covariates=["gPCs"],
    tol=0.00001,
    maxiter=20,
    use_grm_to_fit_null=False,
    overwrite_variance_ratio_file=True,
    run=True,
)

print(step1_results)

Writing BED: 100%|██████████| 1/1 [00:00<00:00, 236.19it/s]

Writing FAM... done.
Writing BIM... done.
[2026-01-09 01:40:06,768] INFO:cellink.tl.external._saigeqtl: Running Step 1: Fitting null model
[2026-01-09 01:40:06,769] INFO:cellink.tl._runner: Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data wzhou88/saigeqtl /app/.pixi/envs/default/bin/Rscript /app/extdata/step1_fitNULLGLMM_qtl.R --phenoFile=gene1_step1_phenotype.txt --phenoCol=ENSG00000273362 --sampleIDColinphenoFile=IND_ID --traitType=count --plinkFile=gene1_step1 --outputPrefix=gene1_step1 --tol=1e-05 --LOCO=FALSE --useGRMtoFitNULL=FALSE --IsOverwriteVarianceRatioFile=TRUE --covarColList=sex,age,gPCs_0,gPCs_1,gPCs_2,gPCs_3,gPCs_4,gPCs_5,gPCs_6,gPCs_7,gPCs_8,gPCs_9,gPCs_10,gPCs_11,gPCs_12,gPCs_13,gPCs_14,gPCs_15,gPCs_16,gPCs_17,gPCs_18,gPCs_19 --sampleCovarColList=sex,age,gPCs_0,gPCs_1,gPCs_2,gPCs_3,gPCs_4,gPCs_5,gPCs_6,gPCs_7,gPCs_8,gPCs_9,gPCs_10,gPCs_11,gPCs_12,gPCs_13,gPCs_14,gPCs_15,gPCs_16,gPCs_17

[2026-01-09 01:40:24,143] INFO:cellink.tl._runner: R version 4.4.3 (2025-02-28)
Platform: x86_64-conda-linux-gnu
Running under: Ubuntu 20.04.4 LTS

Matrix products: default
BLAS/LAPACK: /app/.pixi/envs/default/lib/libopenblasp-r0.3.30.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: NA
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] optparse_1.7.5 SAIGEQTL_0.3.2

loaded via a namespace (and not attached):
[1] compiler_4.4.3      Matrix_1.7-3        Rcpp_1.1.0         
[4] getopt_1.20.4       RcppNumerical_0.6-0 grid_4.4.3         
[7] data.table_1.17.6   RcppParallel_5.1.9  lattice_0.22-

#### Step 2 Only: Association Testing

In [6]:
step2_results = run_saigeqtl(
    dd,
    gene_col="ENSG00000273362",
    prefix="gene1_step1",  # Use same prefix as Step 1
    steps=[2],  # Only Step 2
    mode="cis",
    window=cis_window,
    min_mac=20,
    gmmat_model_file="gene1_step1.rda",  # From Step 1
    variance_ratio_file="gene1_step1.varianceRatio.txt",  # From Step 1
    run=True,
)

step2_results.head()

Writing BED: 100%|██████████| 1/1 [00:00<00:00, 188.01it/s]

Writing FAM... done.
Writing BIM... done.
[2026-01-09 01:40:25,065] INFO:cellink.tl.external._saigeqtl: Running Step 2: Association tests
[2026-01-09 01:40:25,066] INFO:cellink.tl._runner: Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data wzhou88/saigeqtl /app/.pixi/envs/default/bin/Rscript /app/extdata/step2_tests_qtl.R --bedFile=gene1_step1.bed --bimFile=gene1_step1.bim --famFile=gene1_step1.fam --GMMATmodelFile=gene1_step1.rda --varianceRatioFile=gene1_step1.varianceRatio.txt --SAIGEOutputFile=gene1_step1_results.txt --minMAC=20 --LOCO=FALSE --rangestoIncludeFile=gene1_step1_ranges.txt


[2026-01-09 01:40:40,849] INFO:cellink.tl._runner: R version 4.4.3 (2025-02-28)
Platform: x86_64-conda-linux-gnu
Running under: Ubuntu 20.04.4 LTS

Matrix products: default
BLAS/LAPACK: /app/.pixi/envs/default/lib/libopenblasp-r0.3.30.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: NA
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] data.table_1.17.6   optparse_1.7.5      RhpcBLASctl_0.23-42
[4] SAIGEQTL_0.3.2     

loaded via a namespace (and not attached):
[1] compiler_4.4.3      Matrix_1.7-3        Rcpp_1.1.0         
[4] getopt_1.20.4       RcppNumerical_0.6-0 grid_4.4.3         
[7]

,CHR,POS,MarkerID,Allele1,Allele2,AC_Allele2,AF_Allele2,MissingRate,BETA,SE,Tstat,var,p.value,p.value.NA,Is.SPA,N
0,22,16569715,22_16569715_G_A,A,G,106,0.530,0,9.658680e-16,54135.1,3.295800e-25,3.412270e-10,1,1,False,100
1,22,16569887,22_16569887_T_A,A,T,133,0.665,0,7.073560e-16,61436.3,1.874080e-25,2.649420e-10,1,1,False,100
2,22,16570090,22_16570090_ACATT_A,A,ACATT,151,0.755,0,5.051070e-16,69893.4,1.033980e-25,2.047040e-10,1,1,False,100
3,22,16570885,22_16570885_T_G,G,T,141,0.705,0,1.208410e-15,63757.8,2.972680e-25,2.459990e-10,1,1,False,100
4,22,16571233,22_16571233_G_A,A,G,101,0.505,0,5.824050e-16,56733.4,1.809460e-25,3.106870e-10,1,1,False,100


#### Step 3 Only: Gene-Level P-values

In [7]:
# Run only Step 3 (requires Step 2 results)
step3_results = run_saigeqtl(
    dd,
    gene_col="ENSG00000273362",
    prefix="gene1_step1",  # Same prefix
    steps=[3],  # Only Step 3
    gene_name="MYC",  # Optional gene name for output
    run=True,
)

print(step3_results)

[2026-01-09 01:40:40,873] INFO:cellink.tl.external._saigeqtl: Running Step 3: Gene-level p-values
[2026-01-09 01:40:40,875] INFO:cellink.tl._runner: Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data wzhou88/saigeqtl /app/.pixi/envs/default/bin/Rscript /app/extdata/step3_gene_pvalue_qtl.R --assocFile=gene1_step1_results.txt --genePval_outputFile=gene1_step1_gene_pvalue.txt --geneName=MYC
[2026-01-09 01:40:55,516] INFO:cellink.tl._runner: R version 4.4.3 (2025-02-28)
Platform: x86_64-conda-linux-gnu
Running under: Ubuntu 20.04.4 LTS

Matrix products: default
BLAS/LAPACK: /app/.pixi/envs/default/lib/libopenblasp-r0.3.30.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_ME

### 2. Advanced: Custom Covariate Specification
Including cell-type-specific covariates and custom normalizations:

In [8]:
sc.pp.pca(dd.C, n_comps=15)


results_custom = run_saigeqtl(
    dd,
    gene_col="ENSG00000273362",
    prefix="saigeqtl_custom",
    mode="cis",
    window=cis_window,
    sample_covariates=["gPCs"],
    cell_covariates=["X_pca"],
    encode_sex=True,
    encode_age=True,
    min_mac=20,
    use_grm_to_fit_null=False,
    overwrite_variance_ratio_file=True,
    run=True,
)

results_custom.head()

Writing BED: 100%|██████████| 1/1 [00:00<00:00, 242.82it/s]

Writing FAM... done.
Writing BIM... done.
[2026-01-09 01:40:56,880] INFO:cellink.tl.external._saigeqtl: Running Step 1: Fitting null model
[2026-01-09 01:40:56,881] INFO:cellink.tl._runner: Executing: docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data wzhou88/saigeqtl /app/.pixi/envs/default/bin/Rscript /app/extdata/step1_fitNULLGLMM_qtl.R --phenoFile=saigeqtl_custom_phenotype.txt --phenoCol=ENSG00000273362 --sampleIDColinphenoFile=IND_ID --traitType=count --plinkFile=saigeqtl_custom --outputPrefix=saigeqtl_custom --LOCO=FALSE --useGRMtoFitNULL=FALSE --IsOverwriteVarianceRatioFile=TRUE --covarColList=sex,age,gPCs_0,gPCs_1,gPCs_2,gPCs_3,gPCs_4,gPCs_5,gPCs_6,gPCs_7,gPCs_8,gPCs_9,gPCs_10,gPCs_11,gPCs_12,gPCs_13,gPCs_14,gPCs_15,gPCs_16,gPCs_17,gPCs_18,gPCs_19,X_pca_0,X_pca_1,X_pca_2,X_pca_3,X_pca_4,X_pca_5,X_pca_6,X_pca_7,X_pca_8,X_pca_9,X_pca_10,X_pca_11,X_pca_12,X_pca_13,X_pca_14 --sampleCovarColList=sex,age,gPCs_0,g

[2026-01-09 01:41:14,392] INFO:cellink.tl._runner: R version 4.4.3 (2025-02-28)
Platform: x86_64-conda-linux-gnu
Running under: Ubuntu 20.04.4 LTS

Matrix products: default
BLAS/LAPACK: /app/.pixi/envs/default/lib/libopenblasp-r0.3.30.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: NA
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] optparse_1.7.5 SAIGEQTL_0.3.2

loaded via a namespace (and not attached):
[1] compiler_4.4.3      Matrix_1.7-3        Rcpp_1.1.0         
[4] getopt_1.20.4       RcppNumerical_0.6-0 grid_4.4.3         
[7] data.table_1.17.6   RcppParallel_5.1.9  lattice_0.22-

,CHR,POS,MarkerID,Allele1,Allele2,AC_Allele2,AF_Allele2,MissingRate,BETA,SE,Tstat,var,p.value,p.value.NA,Is.SPA,N
0,22,16569715,22_16569715_G_A,A,G,106,0.530,0,-1.287330e-15,54527.0,-4.329770e-25,3.363390e-10,1,1,False,100
1,22,16569887,22_16569887_T_A,A,T,133,0.665,0,-1.732230e-16,61881.1,-4.523640e-26,2.611460e-10,1,1,False,100
2,22,16570090,22_16570090_ACATT_A,A,ACATT,151,0.755,0,-3.843350e-16,70399.5,-7.754820e-26,2.017720e-10,1,1,False,100
3,22,16570885,22_16570885_T_G,G,T,141,0.705,0,-1.066060e-16,64219.4,-2.584940e-26,2.424750e-10,1,1,False,100
4,22,16571233,22_16571233_G_A,A,G,101,0.505,0,-1.350560e-15,57144.1,-4.135900e-25,3.062370e-10,1,1,False,100


## Advanced Usage: Dry Run and Command Generation
### Generate Commands Without Execution
Useful for debugging or running on HPC clusters:

In [9]:
# Generate command strings without executing
commands = run_saigeqtl(
    dd,
    gene_col="ENSG00000273362",
    prefix="saigeqtl_cluster",
    mode="cis",
    window=cis_window,
    sample_covariates=["gPCs"],
    use_grm_to_fit_null=False,
    overwrite_variance_ratio_file=True,
    run=False,  # Don't execute, just return commands
)

print("Step 1 (Null model):")
print(commands["step1"])
print("\nStep 2 (Association tests):")
print(commands["step2"])

Writing BED: 100%|██████████| 1/1 [00:00<00:00, 193.35it/s]

Writing FAM... done.
Writing BIM... done.
Step 1 (Null model):
docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/cellink_data:/cellink_data -w /data wzhou88/saigeqtl /app/.pixi/envs/default/bin/Rscript /app/extdata/step1_fitNULLGLMM_qtl.R --phenoFile=saigeqtl_cluster_phenotype.txt --phenoCol=ENSG00000273362 --sampleIDColinphenoFile=IND_ID --traitType=count --plinkFile=saigeqtl_cluster --outputPrefix=saigeqtl_cluster --LOCO=FALSE --useGRMtoFitNULL=FALSE --IsOverwriteVarianceRatioFile=TRUE --covarColList=sex,age,gPCs_0,gPCs_1,gPCs_2,gPCs_3,gPCs_4,gPCs_5,gPCs_6,gPCs_7,gPCs_8,gPCs_9,gPCs_10,gPCs_11,gPCs_12,gPCs_13,gPCs_14,gPCs_15,gPCs_16,gPCs_17,gPCs_18,gPCs_19 --sampleCovarColList=sex,age,gPCs_0,gPCs_1,gPCs_2,gPCs_3,gPCs_4,gPCs_5,gPCs_6,gPCs_7,gPCs_8,gPCs_9,gPCs_10,gPCs_11,gPCs_12,gPCs_13,gPCs_14,gPCs_15,gPCs_16,gPCs_17,gPCs_18,gPCs_19

Step 2 (Association tests):
docker run --rm -v /Users/larnoldt/sc-genetics/docs/tutorials:/data -v /Users/larnoldt/ce

## Save Commands to File
For batch submission to compute clusters:

In [10]:
commands = run_saigeqtl(
    dd,
    gene_col="ENSG00000273362",
    prefix="saigeqtl_batch",
    mode="cis",
    window=cis_window,
    sample_covariates=["gPCs"],
    run=False,
    use_grm_to_fit_null=False,
    overwrite_variance_ratio_file=True,
    save_cmd_file="saigeqtl_job.sh",
)

Writing BED: 100%|██████████| 1/1 [00:00<00:00, 240.62it/s]

Writing FAM... done.
Writing BIM... done.
